<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/nl_whisper_large_test_(edit).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-1x9c34l1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-1x9c34l1
  Resolved https://github.com/huggingface/transformers to commit d269c4b2d7bbe8f25f3daab818bb13bb5ea4ca45
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.37.0.dev0-py3-none-any.whl size=8269300 sha256=c0f96a679da452851b2fe423c7fbb29604dd0dec0b9457b5d95b207adbc75f9b
  Stored in directory: /tmp/pip-ephem-wheel-cache-2wf4qzrd/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [3]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [4]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "nl", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 1137
})

In [6]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 12.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.3 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801356 sha256=19a5b6fe5b3cfb86b1c27f4fa0ccdedc76e481cfa57a470a1720b59f6de4ead7
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [7]:
model = whisper.load_model("large")

100%|█████████████████████████████████████| 2.88G/2.88G [01:54<00:00, 27.0MiB/s]


In [8]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [9]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [10]:
text

,0
0,en daar die eigen middelen het ook mogelijk ma...
1,je hebt een zeer goede en sterke europese gedr...
2,tegen die achtergrond vindt het voornemen dat ...
3,er is wel een duidelijke europese uitdaging al...
4,nog dramatischer is de situatie in libanon. da...
...,...
1132,de lijst van recente misdaden tegen christenen...
1133,ik wil graag dat de boeren op hun eigen benen ...
1134,extra regels zullen het geschonden consumenten...
1135,ik vind ook dat de wetenschap zoveel mogelijk ...


In [11]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [12]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,male,en daar die eigen middelen het ook mogelijk ma...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00738525390625, 0.00439453125, 0.0005798339...",16000
1,female,je hebt een zeer goede en sterke europese gedr...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000213623046875, 0.000152587890625, -0.0001...",16000
2,male,tegen die achtergrond vindt het voornemen dat ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.02880859375, -0.034271240234375, -0.033355...",16000
3,male,er is wel een duidelijke europese uitdaging al...,/root/.cache/huggingface/datasets/downloads/ex...,"[6.103515625e-05, 0.000152587890625, -6.103515...",16000
4,male,nog dramatischer is de situatie in libanon. da...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, -0.0001220703125, -0.00018...",16000
...,...,...,...,...,...
1132,male,de lijst van recente misdaden tegen christenen...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00103759765625, 0.00103759765625, 0.0012207...",16000
1133,male,ik wil graag dat de boeren op hun eigen benen ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.193695068359375, -0.7314453125, -0.4067687...",16000
1134,male,extra regels zullen het geschonden consumenten...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0008544921875, -0.00079345703125, -0.00012...",16000
1135,male,ik vind ook dat de wetenschap zoveel mogelijk ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001861572265625, 0.00115966796875, 0.00265...",16000


In [13]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
0,male,en daar die eigen middelen het ook mogelijk ma...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00738525390625, 0.00439453125, 0.0005798339...",16000
2,male,tegen die achtergrond vindt het voornemen dat ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.02880859375, -0.034271240234375, -0.033355...",16000
3,male,er is wel een duidelijke europese uitdaging al...,/root/.cache/huggingface/datasets/downloads/ex...,"[6.103515625e-05, 0.000152587890625, -6.103515...",16000
4,male,nog dramatischer is de situatie in libanon. da...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, -0.0001220703125, -0.00018...",16000
5,male,wij als parlement moeten ook niet doen alsof w...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.010894775390625, 0.0115966796875, 0.0105895...",16000
...,...,...,...,...,...
1132,male,de lijst van recente misdaden tegen christenen...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00103759765625, 0.00103759765625, 0.0012207...",16000
1133,male,ik wil graag dat de boeren op hun eigen benen ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.193695068359375, -0.7314453125, -0.4067687...",16000
1134,male,extra regels zullen het geschonden consumenten...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0008544921875, -0.00079345703125, -0.00012...",16000
1135,male,ik vind ook dat de wetenschap zoveel mogelijk ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001861572265625, 0.00115966796875, 0.00265...",16000


In [14]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
1,female,je hebt een zeer goede en sterke europese gedr...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000213623046875, 0.000152587890625, -0.0001...",16000
9,female,ik denk dat het een oogopener moet zijn of ik ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.076171875, -0.060577392578125, -0.03576660...",16000
10,female,deze soort kan tot dertig jaar oud worden.,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00701904296875, 0.002227783203125, -0.00909...",16000
13,female,voorzitter of je nu voor kernernergie bent of ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.010009765625, 0.00469970703125, 0.005584716...",16000
14,female,willen we dat mensen tweederangsburgers zijn v...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0162353515625, 0.031524658203125, 0.0357360...",16000
...,...,...,...,...,...
1125,female,dat kan natuurlijk.,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000244140625, -0.0001220703125, -6.1035156...",16000
1126,female,laten we elkaar vooral aan deze gemaakte afspr...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0003662109375, 0.000457763671875, 0.000701...",16000
1128,female,je ziet in veel landen dat er een quotasysteem...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.039276123046875, -0.052520751953125, -0.07...",16000
1129,female,ja dit parlement debatteert met emotie en nee ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.01861572265625, -0.033599853515625, -0.048...",16000


In [15]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="nl", fp16=False)
    return result["text"]

In [16]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [17]:
from jiwer import wer
from collections import Counter

def calculate_precision_recall_f1(ground_truth, hypothesis):
    ground_truth_words = set(ground_truth.split())
    hypothesis_words = set(hypothesis.split())

    true_positives = ground_truth_words.intersection(hypothesis_words)
    false_positives = hypothesis_words - ground_truth_words
    false_negatives = ground_truth_words - hypothesis_words

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if true_positives or false_positives else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if true_positives or false_negatives else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall else 0

    return precision, recall, f1_score

In [18]:

total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)

    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / num_sample
average_precision = total_precision / num_sample
average_recall = total_recall / num_sample
average_f1 = total_f1 / num_sample

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 0.2, Precision: 0.8333333333333334, Recall: 0.8, F1: 0.816326530612245
Processed 2 files, WER: 0.13636363636363635, Precision: 0.8529411764705882, Recall: 0.8529411764705882, F1: 0.8529411764705882
Processed 3 files, WER: 0.2, Precision: 0.8, Recall: 0.8, F1: 0.8000000000000002
Processed 4 files, WER: 0.3684210526315789, Precision: 0.625, Recall: 0.5882352941176471, F1: 0.6060606060606061
Processed 5 files, WER: 0.21428571428571427, Precision: 0.8571428571428571, Recall: 0.9230769230769231, F1: 0.888888888888889
Processed 6 files, WER: 0.18421052631578946, Precision: 0.7941176470588235, Recall: 0.7941176470588235, F1: 0.7941176470588235
Processed 7 files, WER: 0.1111111111111111, Precision: 0.9375, Recall: 0.8823529411764706, F1: 0.9090909090909091
Processed 8 files, WER: 0.35, Precision: 0.631578947368421, Recall: 0.6666666666666666, F1: 0.6486486486486486
Processed 9 files, WER: 1.0, Precision: 0.0, Recall: 0.0, F1: 0
Processed 10 files, WER: 0.2, Precision: 0

In [19]:
total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)


    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / num_sample
average_precision = total_precision / num_sample
average_recall = total_recall / num_sample
average_f1 = total_f1 / num_sample

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 0.3076923076923077, Precision: 0.75, Recall: 0.84, F1: 0.7924528301886793
Processed 2 files, WER: 1.0, Precision: 0.0, Recall: 0.0, F1: 0
Processed 3 files, WER: 0.375, Precision: 0.625, Recall: 0.625, F1: 0.625
Processed 4 files, WER: 0.36, Precision: 0.75, Recall: 0.8181818181818182, F1: 0.7826086956521738
Processed 5 files, WER: 0.25, Precision: 0.7307692307692307, Recall: 0.7916666666666666, F1: 0.76
Processed 6 files, WER: 0.35294117647058826, Precision: 0.7333333333333333, Recall: 0.6470588235294118, F1: 0.6875
Processed 7 files, WER: 0.25, Precision: 0.7692307692307693, Recall: 0.8333333333333334, F1: 0.8
Processed 8 files, WER: 0.2222222222222222, Precision: 0.7777777777777778, Recall: 0.8235294117647058, F1: 0.7999999999999999
Processed 9 files, WER: 0.5, Precision: 0.6666666666666666, Recall: 0.6666666666666666, F1: 0.6666666666666666
Processed 10 files, WER: 0.3, Precision: 0.76, Recall: 0.7307692307692307, F1: 0.7450980392156863
Processed 11 files, W